# Building a CrewAI Agent with Long-Term Memory via Flotorch

This notebook demonstrates how to build an advanced CrewAI agent with both short-term and long-term memory capabilities using the Flotorch platform. This allows the agent to recall information not only from the current conversation but also from previous, separate interactions.

### Prerequesit
Configure model, memory provider and API key in Flotroch console (https://console.flotorch.cloud/)

### Viewing logs
Logs can be viewed in logs tab in Flotroch console (https://console.flotorch.cloud/)

### Key Concepts:
- **Short-Term Memory**: Provided by `FlotorchCrewAISession` for session-specific context.
- **External (Long-Term) Memory**: Provided by `FlotorchMemoryStorage` for persistent knowledge across all sessions.
- **`Agent`, `Task`, `Crew`**: Core CrewAI components configured to leverage both memory types.

## 1. Setup and Imports

The following cells install the necessary packages, configure API credentials, and import the required components from Flotorch and CrewAI, including those needed for external memory.

In [ ]:
# install flotorch adk package
%pip install flotorch[crewai]

In [ ]:
%pip install crewai crewai_tools crewai_tools[mcp]

In [ ]:
FLOTORCH_API_KEY = "<flotorch api key>"
FLOTORCH_BASE_URL = "<flotroch gateway base url>" # eg: https://gateway.flotorch.cloud"
FLOTORCH_MODEL = "<flotorch model>"
USER_ID = "flotorch_user_001"
APP_ID = "flotorch_app_001"
MEMORY_PROVIDER = "<flotorch memory provider>"

In [ ]:
# Import CrewAI and related modules
import datetime
from flotorch.crewai.llm import FlotorchCrewAILLM
from flotorch.crewai.sessions import FlotorchCrewAISession
from flotorch.crewai.memory import FlotorchMemoryStorage
from crewai import Agent, Task, Crew
from crewai.memory.short_term.short_term_memory import ShortTermMemory
from crewai.memory.external.external_memory import ExternalMemory

print("Imported necessary libraries successfully")


## 2. Model Configuration

We initialize the `FlotorchCrewAILLM` which will act as the brain for our agent, enabling it to process information and make decisions.

In [ ]:
model  =  FlotorchCrewAILLM(
        model_id=FLOTORCH_MODEL,
        api_key=FLOTORCH_API_KEY,
        base_url=FLOTORCH_BASE_URL,
    )

print(f"Flotorch LLM model configured: {model.model}")

## 3. Memory Configuration

This is where we set up the agent's memory systems. We configure both short-term and long-term memory to give the agent a comprehensive ability to recall information.

- **Short-Term Memory**: Uses `FlotorchCrewAISession` to remember the current conversation's flow.
- **External Memory**: Uses `FlotorchMemoryStorage` to store and retrieve key information across different sessions, acting as the agent's long-term knowledge base.

In [ ]:
short_term_storage = FlotorchCrewAISession(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL
)

short_term_memory = ShortTermMemory(storage = short_term_storage)

external_storage = FlotorchMemoryStorage(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    name = MEMORY_PROVIDER,
    user_id = USER_ID,
    app_id  = APP_ID
)

external_memory = ExternalMemory(storage = external_storage)

print("Initialized short_term and external Memory")

## 4. Agent and Task Configuration

We define the `Agent` and `Task` with a goal that explicitly instructs it to use its memory capabilities to provide accurate and context-aware responses.

In [ ]:
# Create the Agent
agent = Agent(
    role="Conversational Memory Agent",
    goal=""" You are a helpful conversational agent with short-term memory. and external memory
    You remember recent conversation context to give better answers.
    You respond naturally to user queries while recalling relevant past information.
    user_query: {query}""",
    backstory=(
        "An engaging assistant who remembers recent interactions "
        "and uses them to answer questions accurately and conversationally."
    ),
    allow_delegation=False,
    verbose=False,
    llm=model
)

# Create the Task (goal is same as agent)
task = Task(
    description=""" You are a helpful conversational agent with short-term memory and external memory.
    You remember recent conversation context to give better answers.
    You respond naturally to user queries while recalling relevant past information.
    user_query: {query}""",
    expected_output=(
        "A natural, conversational answer that may use recent memory if relevant."
    ),
    agent=agent
)
# Create custom tools and agent
print(f"Agent and Task created successfully: {agent.role}")

## 5. Crew Assembly with Memory

The `Crew` is assembled with the agent, task, and crucially, both the `short_term_memory` and `external_memory` modules. This equips the crew with a full range of memory capabilities.

In [ ]:
crew = Crew(
    agents = [agent],
    tasks = [task],
    short_term_memory = short_term_memory,
    external_memory = external_memory,
    verbose = False

)

print("Created Crew with agent,task and memory")

## 6. Interactive Chat

Start a chat session to interact with the agent. Test its ability to recall information you provide, both within this conversation (short-term) and from facts it may have learned in previous runs (long-term). Type 'exit' to end.

In [ ]:
while True:

    user_query = input("user: ")

    if user_query.lower().strip() == "exit":
        break
    response = crew.kickoff(inputs = {"query":user_query})
    print(f"Assistant:{response.raw}")

## Summary

This notebook detailed the implementation of a sophisticated AI agent equipped with both short-term and long-term memory.  
By leveraging **Flotorch's infrastructure**, we created an agent that not only remembers the current conversation but also retains and recalls information across separate sessions.

### Key Achievements

- **Dual-Memory Architecture**  
  Successfully integrated two distinct memory types:  
  - `FlotorchCrewAISession` for immediate, short-term conversational context.  
  - `FlotorchMemoryStorage` for persistent, long-term knowledge.

- **Enhanced Agent Intelligence**  
  The agent demonstrated its ability to recall facts from a knowledge base established in previous interactions, leading to more intelligent and informed responses.

- **Comprehensive Solution**  
  This dual-memory approach provides a powerful template for building advanced agents capable of complex, ongoing, and context-rich dialogues.
